# Install MONAA

Download, build, and install  [MONAA](https://github.com/MasWag/monaa)

In [0]:
! git clone https://github.com/MasWag/monaa.git

Cloning into 'monaa'...
remote: Enumerating objects: 400, done.
remote: Counting objects: 100% (400/400), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 1193 (delta 276), reused 362 (delta 253), pack-reused 793
Receiving objects: 100% (1193/1193), 1.32 MiB | 10.87 MiB/s, done.
Resolving deltas: 100% (780/780), done.


In [0]:
! apt install cmake libboost-dev flex libeigen3-dev doxygen bison -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
cmake is already the newest version (3.10.2-1ubuntu2).
doxygen is already the newest version (1.8.13-10).
flex is already the newest version (2.6.4-6).
libboost-dev is already the newest version (1.65.1.0ubuntu1).
libeigen3-dev is already the newest version (3.3.4-4).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libbison-dev
Suggested packages:
  bison-doc
The following NEW packages will be installed:
  bison libbison-dev
0 upgraded, 2 newly installed, 0 to remove and 7 not upgraded.
Need to get 605 kB of archives.
After this operation, 1,811 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libbison-dev amd64 2:3.0.4.dfsg-1build1 [339 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 bison

In [0]:
! mkdir -p monaa/build && cd monaa/build && cmake -DCMAKE_BUILD_TYPE=Release .. && make -j4  && make install

-- Found BISON: /usr/bin/bison (found version "3.0.4") 
-- Found FLEX: /usr/bin/flex (found version "2.6.4") 
-- Boost version: 1.65.1
-- Found the following Boost libraries:
--   program_options
--   unit_test_framework
--   iostreams
--   graph
--   regex
-- Found Doxygen: /usr/bin/doxygen (found version "1.8.13") found components:  doxygen dot 
-- Configuring done
-- Generating done
-- Build files have been written to: /content/monaa/build
[ 20%] [BISON][tre_parser] Building parser with bison 3.0.4
[ 20%] [FLEX][tre_lexer] Building scanner with flex 2.6.4
monaa/tre_parser.yy: warning: 21 shift/reduce conflicts [-Wconflicts-sr]
monaa/tre_lexer.l:51: warning, -s option given but default rule can be matched
Scanning dependencies of target monaa
[ 30%] Building CXX object CMakeFiles/monaa.dir/libmonaa/ta2za.cc.o
[ 40%] Building CXX object CMakeFiles/monaa.dir/monaa/main.cc.o
[ 50%] Building CXX object CMakeFiles/monaa.dir/monaa/tre.cc.o
[ 60%] Building CXX object CMakeFiles/monaa.dir/li

# Try the sample example (small.txt, small.dot)

In [0]:
%cat ./monaa/examples/small.txt

h       0.1
l       0.3
l       0.5
h       0.7


In [0]:
%cat ./monaa/examples/small.dot

digraph G{
        1 [init=1][match=0]
        2 [init=0][match=0]
        3 [init=0][match=0]
        4 [init=0][match=1]
        1->2 [label=l][guard="{x0 < 1}"];
        2->3 [label=h][guard="{x0 < 1}"];
        3->2 [label=l][guard="{x0 < 1}"];
        3->4 [label="$"][guard="{x0 < 1}"];
}



In [0]:
! monaa -f ./monaa/examples/small.dot < ./monaa/examples/small.txt

  0.300000       <= t <   0.500000
  0.700000        < t' <   1.500000
  0.200000        < t' - t <   1.000000


# Try custom example (example on the poster)

In [0]:
%%writefile /tmp/custom.txt
A    1
B    8
A    15
B    20
A    23
B    27
A    30
B    34
A    40

Writing /tmp/custom.txt


## Using Timed regular expression

In [0]:
! monaa -e '(ABAB(AB)+)%(0,20)$' < /tmp/custom.txt

  8.000000       <= t <  15.000000
 34.000000        < t' <=  40.000000
 19.000000        < t' - t <=  32.000000


## Using Timed Automaton

In [0]:
%%writefile /tmp/custom.dot
digraph G {
    start [init=1][match=0]
    A1 [init=0][match=0]
    B1 [init=0][match=0]
    A2 [init=0][match=0]
    B2 [init=0][match=0]
    A3 [init=0][match=0]
    B3 [init=0][match=0]
    end [init=0][match=1]
    
    start -> A1 [label=A][reset="{0}"]
    A1 -> B1 [label=B]
    B1 -> A2 [label=A]
    A2 -> B2 [label=B]
    B2 -> A3 [label=A]
    A3 -> B3 [label=B][guard="{x0 < 20}"]
    B3 -> A3 [label=A]
    B3 -> end [label="$"]
}

Overwriting /tmp/custom.dot


In [0]:
! monaa -f /tmp/custom.dot< /tmp/custom.txt

  8.000000       <= t <  15.000000
 34.000000        < t' <=  40.000000
 19.000000        < t' - t <=  32.000000
